In [1]:
import model
import pickle
import os

def newNeuralNetwork():
    return model.NeuralNetwork((70, 20), "rapid", 0.1, 64)

folder_name = "process-output_2"
if len(os.listdir(folder_name)) > 0:
    raise Ex

In [2]:
class Node:
    
    def __init__(self, state):
        self.state = state
        self.next = []
        
    def connect(self, node, operation_type, operation):
        self.next.append((operation_type, operation, node))

In [3]:
def forkSteepnessIncrease(nn, node):
    
    # Save parameters
    old_params = nn.getParams()
    
    current_node = node
    
    for step in range(8):
        
        # Increase steepness
        steepening_op = nn.steepen()
        next_node = Node(nn.getState())
        current_node.connect(next_node, "steepening", steepening_op)
        current_node = next_node
        
        # Train the model
        training_op = nn.train(100)
        next_node = Node(nn.getState())
        current_node.connect(next_node, "training", training_op)
        current_node = next_node
        
    nn.setParams(old_params)

In [4]:
main_nn = newNeuralNetwork()

root_node = Node(main_nn.getState())
current_node = root_node

for stage in range(30):
    
    # Train the model
    training_op = main_nn.train(100)
    next_node = Node(main_nn.getState())
    current_node.connect(next_node, "training", training_op)
    current_node = next_node
    
    # Make a fork from the main model and increase steepness
    forkSteepnessIncrease(main_nn, current_node)
    
    # Increase sparseness
    sparsening_op = main_nn.sparsen()
    next_node = Node(main_nn.getState())
    current_node.connect(next_node, "sparsening", sparsening_op)
    current_node = next_node
    
    with open("process-output_1/intermediate-{}.pickle".format(stage), "wb") as output_file:
        pickle.dump({"root": root_node}, output_file)
    
# Train the model
training_op = main_nn.train(100)
next_node = Node(main_nn.getState())
current_node.connect(next_node, "training", training_op)
current_node = next_node

# Make a fork from the main model and increase steepness
forkSteepnessIncrease(main_nn, current_node)

In [5]:
with open("model.py", "r") as source_file:
    model_source = source_file.read()

with open("process-output_1/final.pickle", "wb") as output_file:
    pickle.dump({"root": root_node, "model_source": model_source}, output_file)

In [6]:
root_node